# Practice Hackathon

In [ ]:
#Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn import datasets as dat
from sklearn import tree
from sklearn import metrics as met
from sklearn import model_selection as mod
from sklearn import linear_model as lin
from sklearn import preprocessing as pre
from sklearn import pipeline as pip # sklearn pipeline
from sklearn import svm
from sklearn import impute as imp
from sklearn.experimental import enable_iterative_imputer
from sklearn import compose as com
from sklearn import ensemble as ens

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Data read from file

df_data = pd.read_csv("/content/final_data_ivaa.csv",sep=",")
df_data.head(3)

In [ ]:
df_data.info()

In [ ]:
#Data columns drop

df_data = df_data.drop(columns=['Kategorı','Register_date'])

In [ ]:
#Identify the unique values

pd.set_option('display.max_rows', None)
dict = {}

for i in list(df_data.columns):
  dict[i] = df_data[i].value_counts().shape[0]

uniq = pd.DataFrame(dict, index = ["Unique counts"]).transpose()
uniq.sort_values(by="Unique counts")

In [ ]:
#Unique values

data["Colum_Name"].unique()

In [ ]:
for i in range(len(df_data.columns)):
  df_data[df_data.columns[i]].fillna(0, inplace=True)

In [ ]:
#Finding Data has null value or not

df_data.isnull().sum()

In [ ]:
#Split data to train and test

X_train, X_test, y_train, y_test = mod.train_test_split(
    df_data.drop("CASE_DURUM", axis=1), df_data["CASE_DURUM"], test_size=0.33, random_state=42)

In [ ]:
#categorical colums

cat_columns = list(X_train.select_dtypes(include="object").columns)
cat_columns

In [ ]:
#numerical colums
num_columns = list(X_train.select_dtypes(exclude="object").columns)
num_columns

In [ ]:
#Categoricial colums distribution

for col in cat_columns:
  if len(X_train[col].unique()) < 200:
    X_train[col].value_counts().plot(kind="bar", figsize=(10, 6))
    plt.title(col)
    plt.show()

In [ ]:
#Numericial colums distribution

for col in num_columns:
  sns.distplot(X_train[col])
  plt.show()

In [ ]:
# Pipeline with İmputer and Column Transformasyon

#ct_num = com.ColumnTransformer([
#    ("num_skewed", pre.FunctionTransformer(np.log1p), num_columns)
#])

pipe_num = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("imp", imp.SimpleImputer(strategy="mean"))
])

pipe_cat = pip.Pipeline([
    ("imp", imp.SimpleImputer(strategy="most_frequent")),
    ("encoding", pre.OneHotEncoder(handle_unknown="ignore", sparse=True)) 
])

ct = com.ColumnTransformer([
    ("num", pipe_num, num_columns),
    ("cat", pipe_cat, cat_columns)
])

ct.fit_transform(X_train)

In [ ]:
# SGDClassifier
pipe_all = pip.Pipeline([
    ("ct", ct),
    ("sgd", lin.SGDClassifier(random_state=42))
])

pipe_all.fit(X_train, y_train)

In [ ]:
# LogisticRegression

pipe_LG = pip.Pipeline([
    ("ct", ct),
    ("log_reg", lin.LogisticRegression(random_state=42))
])

pipe_LG.fit(X_train, y_train)

In [ ]:
mod.cross_val_score(pipe_LG, X_train, y_train, cv=3, scoring="accuracy").mean()
pipe_LG.fit(X_train, y_train)
y_pred = pipe_LG.predict(X_test)
print(met.classification_report(y_test, y_pred))

In [ ]:
# Decision Tree

pipe_DT = pip.Pipeline([
    ("ct", ct),
    ("DT", tree.DecisionTreeClassifier(max_depth=6, min_samples_split=5,random_state=42))
])

pipe_DT.fit(X_train, y_train)
y_pred = pipe_DT.predict(X_test)
met.accuracy_score(y_test, y_pred)

mod.cross_val_score(pipe_DT, X_train, y_train, cv=5, scoring="accuracy").mean()


In [ ]:
# Learning Curve

def plotLearningCurve(est, X_train, y_train, X_test, y_test, n_iter=10, minY=0, maxY=1, useF1Score=False):
  x_values = []
  y_values_train = []
  y_values_test = []

  for i in np.linspace(100, len(X_train), n_iter):
    i = int(i)

    # select a subset of training data
    X_train_temp = X_train[:i]
    y_train_temp = y_train[:i]

    # create the model
    est.fit(X_train_temp, y_train_temp)

    # evaluate train set
    y_pred_train = est.predict(X_train_temp)
    if useF1Score:
      train_score = met.f1_score(y_train_temp, y_pred_train)
    else:
      train_score = met.accuracy_score(y_train_temp, y_pred_train)

    # evaluate test set
    y_pred_test = est.predict(X_test)
    if useF1Score:
      test_score = met.f1_score(y_test, y_pred_test)
    else:
      test_score = met.accuracy_score(y_test, y_pred_test)

    # populate lists
    y_values_train.append(train_score)
    y_values_test.append(test_score)
    x_values.append(i)
  
  myLabel = "Accuracy"
  if useF1Score:
    myLabel = "F1-Score"
  plt.figure(figsize=(10, 6))
  plt.plot(x_values, y_values_train, label="train")
  plt.plot(x_values, y_values_test, label="test")
  plt.legend()
  plt.ylabel(myLabel)
  plt.xlabel("# of training samples")
  plt.grid(True)
  plt.ylim(minY, maxY)
  plt.show()

In [ ]:
plotLearningCurve(pipe_DT,
                  X_train, y_train, X_test, y_test, 
                  n_iter=10, minY=0.6, maxY=0.72, useF1Score=False)

In [ ]:
i = 0
list_of_Class = ["SGDClassifier","DecisionTreeClassifier","LogisticRegression"]
for est in [pipe_all, 
            pipe_DT,
            pipe_LG]: 

  pipe = pip.Pipeline([
        ("est", est)
  ])

  if hasattr(est, "predict_proba"):
    myMethod = "predict_proba"
  else:
    myMethod = "decision_function"

  y_scores = mod.cross_val_predict(pipe, X_train, y_train,
                                   cv=5, method=myMethod)
  if myMethod == "predict_proba":
    y_scores = y_scores[:, 1] # probabilities for True class

  fpr, tpr, thresholds = met.roc_curve(y_train, y_scores)
  auc_score = met.roc_auc_score(y_train, y_scores)

  print(list_of_Class[i] + ":", auc_score)

  plt.plot(fpr, tpr, label=list_of_Class[i])
  i = i + 1

plt.title("ROC Curve")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.legend()
plt.show()

# Machine Learning By IA

Some supervised learning methods: Linear Regression, Logistic Regression ,Decision Tree, Random Forests, SVMs (Support Vector Machines), ANNs (Artificial Neural Networks)


Some unsupervised learning methods: Clustering → KMeans, Agglomerative Clustering, DBSCAN, etc.

Dimensionality Reduction → Principal Component Analysis (PCA), t-Distributed Stochastic Neighbor Embedding (t-SNE)

Association Rule Mining → Apriori algorithm

Anomaly Detection → One-class SVM 

Overfitting: 1. increase the amount of training data 2. Reduce the number of features 3. Regularization

Underfitting: 1. Try to use a more complex algorithm with more parameter 2. Increase the number of features 3. Reduce regularization


Note: Before start to split your data, ALWAYS do shuffing.

## Data PreProcessing

In [ ]:
#reading data from file
housing = pd.read_csv("/content/housing.csv")
housing.head()

In [ ]:
#data frame shape
housing.shape

In [ ]:
housing.isnull().sum()

In [ ]:
housing.dtypes

In [ ]:
#Below shows the unique labels
housing["ocean_proximity"].unique()

In [ ]:
#target y distribution on boxpot
sns.boxplot(y)

In [ ]:
#target y distribution on distplot
sns.distplot(y)

In [ ]:
#target y distribution on train data
y_train.value_counts().plot(kind="bar")

## Linear Regression

**Simple linear regression :** a single independent variable is used to predict the value of a dependent variable.
Equation : y=A+BX


**Multiple linear regression :** two or more independent variables are used to predict the value of a dependent variable. The difference between the two is the number of independent variables. Equation : y=A+BX1+CX2+DX3

We need to find θ which minimizes the cost function (RMSE or MSE). So Using Normal Equation (Scikit-Linear Regression - No Scalling) and Using Gradient Descent (Scikit-SGDRegressor - Scalling)


**Gradient Descent:** With this algorithm, we must use StandardScaler

*  **Batch Gradient Descent :** “Eta” is the “learning rate” here. It involves calculations over the full training set X, at each Gradient Descent step! As a result it is terribly slow on very large training sets. To find a good learning rate, you can use grid search.
*  **Stochastic Gradient Descent :** Stochastic Gradient Descent just picks a random instance in the training set at every step and computes the gradients based only on that single instance. So this makes the algorithm much faster since it has very little data to manipulate at every iteration.

*  **Mini-batch Gradient Descent :** Mini-batch GD computes the gradients on small random sets of instances called mini- batches.

**Polynomial Regression:** Data is actually more complex. In order to see your assumtion is not overfittin or underfitting, we must check with 1. Cross-Validation 2. Learning Curves


### Linear Regression Algorithms

####Library Package

In [ ]:
#Adding Library Packages
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
#from sklearn.linear_model import LinearRegression
from sklearn import linear_model as lin
from sklearn import preprocessing as pre
from sklearn import metrics as met
from sklearn import model_selection as mod
from sklearn import pipeline as pip

###Distrubution of Y data

In [ ]:
#target y distribution on distplot
sns.distplot(y)

In [ ]:
#target y distribution on boxpot
sns.boxplot(y)

####Batch Gradient Descent

In [ ]:
m = 200
np.random.seed(42)
X = np.random.rand(m, 1) * 6 - 3 # [-3, 3)
y = 3 * X + 4 + 3*np.random.rand(m, 1)

ones_vect = np.ones((m, 1))
X_2 = np.concatenate((ones_vect, X), axis=1)

for eta in [0.001, 0.1, 10]:
  # randomly initialize theta values
  theta = np.random.rand(2, 1)

  #eta = 0.1
  n_steps = 15

  for i in range(n_steps):
    # compute the gradient
    gradient = (2/m) * X_2.T.dot(X_2.dot(theta) - y)
    # update theta values
    theta = theta - eta*gradient

    # make prediction with current theta values
    y_pred = X_2.dot(theta)
    
    if i == 0:
      myColor = "green"
    elif i == n_steps - 1:
      myColor = "red"
    else:
      myColor = "orange"
    plt.scatter(X, y_pred, color=myColor)

  plt.scatter(X, y, color="blue")
  plt.title("Eta:" + str(eta))
  plt.show()

#### Stochastic Gradient Descent (SGD)

In [ ]:
#data is split to train and test data
X_train, X_test, y_train, y_test = mod.train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#pipeline 
pipe = pip.Pipeline([
        ("scaler", pre.StandardScaler()),              #Scaling
        ("sgd", lin.SGDRegressor(random_state=42))     #Model_Learning_Method
])

pipe.fit(X_train, y_train.ravel())                     #Model_Learning_Fitted

y_pred = pipe.predict(X_test)                          #Prediction

met.mean_squared_error(y_test, y_pred, squared=False)  #RMSE score

mod.cross_val_score(pipe, X_train, y_train.ravel(), cv=3, scoring="neg_root_mean_squared_error").mean() * -1   #Cross Validation Score

In [ ]:
# GridSearchCV
myParams = {
    "sgd__alpha": [0.001, 0.01, 0.1],
    "sgd__penalty": ["l1", "l2"],
    "sgd__eta0": [0.01, 0.1, 1],
    "sgd__max_iter": [10000, 50000]
}

grid = mod.GridSearchCV(pipe, myParams, scoring="neg_root_mean_squared_error", cv=3)
grid.fit(X_train, y_train.ravel())
grid.best_params_     #Best parameters

In [ ]:
grid.best_score_      #Best scores

In [ ]:
# Prediction for best parameters
y_pred = grid.best_estimator_.predict(X_test)
met.mean_squared_error(y_test, y_pred, squared=False)

####Polynomial Data

In [ ]:
#data
m = 500
X = np.random.rand(m, 1) * 6 - 3
y = 3 * X**2 + 4 * X + 5 + 4*np.random.rand(m, 1)

In [ ]:
#data is split to train and test data
X_train, X_test, y_train, y_test = mod.train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
pipe = pip.Pipeline([
      ("poly", pre.PolynomialFeatures(degree=2)),    #polinomal regulazation
      ("scaler", pre.StandardScaler()),              #Scaling
      ("sgd", lin.SGDRegressor(random_state=42))     #Model_Learning_Method
])

pipe.fit(X_train, y_train.ravel())

y_pred = pipe.predict(X_test)

plt.scatter(X_train, y_train)
plt.scatter(X_test, y_pred)
plt.show()

####Learning Curves

In [ ]:
np.linspace(10, len(X_train), 10)

In [ ]:
def plotLearningCurve(est, X_train, y_train, X_test, y_test, n_iter=10, minY=0, maxY=1):
  x_values = []
  y_values_train = []
  y_values_test = []

  for i in np.linspace(10, len(X_train), n_iter):
    i = int(i)

    # select a subset of training data
    X_train_temp = X_train[:i]
    y_train_temp = y_train[:i]

    # create the model
    est.fit(X_train_temp, y_train_temp)

    # evaluate train set
    y_pred_train = est.predict(X_train_temp)
    train_score = met.mean_squared_error(y_train_temp, y_pred_train, squared=False)

    # evaluate test set
    y_pred_test = est.predict(X_test)
    test_score = met.mean_squared_error(y_test, y_pred_test, squared=False)

    # populate lists
    y_values_train.append(train_score)
    y_values_test.append(test_score)
    x_values.append(i)
  
  plt.figure(figsize=(10, 6))
  plt.plot(x_values, y_values_train, label="train")
  plt.plot(x_values, y_values_test, label="test")
  plt.legend()
  plt.ylabel("RMSE")
  plt.xlabel("# of training samples")
  plt.grid(True)
  plt.ylim(minY, maxY)
  plt.show()

In [ ]:
#learning Curve for seeing overfitting and underfitting conditions
plotLearningCurve(pipe, X_train, y_train.ravel(), X_test, y_test.ravel(), n_iter=10, minY=0, maxY=10)

## Logistic Regression

**Sigmoid Function:** it (referred with 𝝈) is a S shaped function, which always output a numeric value between 0 and 1.

**Log Loss:** The cost function. For the cost function, cross entropy is used during the training.


**The hyperparameter:** It is C. The higher the value of C, the less the model is regularized.

**Multiclass Classification :**

*   Binary classification → 2 classes
*   Multiclass classification → Strictly more than 2 classes

**Note :** First, the model calculates the logit scores for each class. Than it uses softmax function to calculate probability of each class.

In LogisticRegression function, we need to set multi_class hyperparameter to "multinomial" to switch it to Softmax Regression. Additionally, solver parameter should be "lbfgs".

### Logistic Regression Algorithms

####Library Package

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import datasets as dat
from sklearn import model_selection as mod
from sklearn import preprocessing as pre
from sklearn import pipeline as pip
from sklearn import linear_model as lin
from sklearn import metrics as met

####Single Classification Model

In [ ]:
#Iris data load
data_obj = dat.load_iris()
X = data_obj.data
y = data_obj.target

In [ ]:
y_binary = (y==2)

In [ ]:
X_train, X_test, y_train, y_test = mod.train_test_split(
    X, y_binary, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
pipe = pip.Pipeline([
        ("scaler", pre.StandardScaler()),
        ("log_reg", lin.LogisticRegression(random_state=42))
])

y_pred = mod.cross_val_predict(pipe, X_train, y_train, cv=3, method="predict")

In [ ]:
#predict the target propability for each label
mod.cross_val_predict(pipe, X_train, y_train, cv=3, method="predict_proba")
np.set_printoptions(suppress=True)
y_proba = pipe.predict_proba(X_test)

In [ ]:
mat = met.confusion_matrix(y_train, y_pred)
mat

In [ ]:
sns.heatmap(mat, annot=True, cmap="RdBu_r")

In [ ]:
print(met.classification_report(y_train, y_pred))

####Multiclass Classification Model

In [ ]:
pipe = pip.Pipeline([
        ("scaler", pre.StandardScaler()),
        ("log_reg", lin.LogisticRegression(multi_class="multinomial",
                                           solver="lbfgs",
                                           random_state=42))
])

params = {
    "log_reg__penalty": ["l2", "none"],
    "log_reg__C": [0.1, 1, 10]
}

In [ ]:
grid = mod.GridSearchCV(pipe, params, scoring="accuracy", cv=3)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
met.accuracy_score(y_test, y_pred)

## Decision Trees

*   Decision Trees are both used for **Regression** and **Classification** tasks.
*   Decision Trees do not require scaling.
*   A control parameter is **max_depth**.
*   **Gini** attribute continoues until all the sample in the node belongs to same class where **Gini = 0**.
*   **min_samples_split:** the minimum number of samples a node must have before it can be split.
*   **min_samples_leaf:** the minimum number of samples a leaf node must have.





###Decision Trees Algorithms 

####Library Package

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import datasets as dat
from sklearn import model_selection as mod
from sklearn import preprocessing as pre
from sklearn import pipeline as pip
from sklearn import linear_model as lin
from sklearn import metrics as met
from sklearn import tree

from sklearn import ensemble as ens

####Algorithms

In [ ]:
data_obj = dat.load_iris()
X = data_obj.data
y = data_obj.target

In [ ]:
X_train, X_test, y_train, y_test = mod.train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
dct_tree = tree.DecisionTreeClassifier(max_depth=2, random_state=42)
dct_tree.fit(X_train, y_train)

In [ ]:
tree.export_graphviz(
        dct_tree,
        out_file="iris_tree.dot",
        feature_names=data_obj.feature_names,
        class_names=data_obj.target_names,
        rounded=True,
        filled=True
    )

In [ ]:
! dot -Tpng iris_tree.dot -o iris_tree.png

In [ ]:
y_train_binary = (y_train == 2)
y_test_binary = (y_test == 2)

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
pd.Series(y_train).value_counts().plot(kind="bar")

In [ ]:
pd.Series(y_train_binary).value_counts().max() / len(y_train_binary)

In [ ]:
dct_tree = tree.DecisionTreeClassifier(random_state=42)

params = {
    "max_depth": [4, 5, 6],
    "min_samples_split": [2, 5, 10]
}

grid = mod.GridSearchCV(dct_tree, params, cv=3, scoring="accuracy")
grid.fit(X_train, y_train_binary)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
y_pred[:5]

In [ ]:
y_proba = grid.best_estimator_.predict_proba(X_test)
y_proba[:5]

In [ ]:
precision, recall, thresholds = met.precision_recall_curve(y_test_binary, y_pred)

In [ ]:
print("precision", len(precision))
print("recall", len(recall))
print("thresholds", len(thresholds))

In [ ]:
met.roc_auc_score(y_test_binary, y_pred)

In [ ]:
for est in [tree.DecisionTreeClassifier(random_state=42), 
            lin.SGDClassifier(random_state=42),
            ens.RandomForestClassifier(random_state=42)]:
  pipe = pip.Pipeline([
        ("scaler", pre.StandardScaler()),
        ("est", est)
  ])

  if hasattr(est, "predict_proba"):
    myMethod = "predict_proba"
  else:
    myMethod = "decision_function"

  y_scores = mod.cross_val_predict(pipe, X_train, y_train_binary,
                                   cv=5, method=myMethod)
  if myMethod == "predict_proba":
    y_scores = y_scores[:, 1] # probabilities for True class
  
  fpr, tpr, thresholds = met.roc_curve(y_train_binary, y_scores)
  auc_score = met.roc_auc_score(y_train_binary, y_scores)
  print(est.__class__.__name__, auc_score)

  plt.plot(fpr, tpr, label=est.__class__.__name__)

plt.title("ROC Curve")
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.legend()
plt.show()

## Imbalanced Learning

*  One common approach for the imbalanced learning is resampling.
*  **Undersampling :** Removes random records from majority class
    - RandomUnderSampler
    - TomekLinks
*  **Oversampling :** Duplicates random records from minority class
    - RandomOverSampler
    - SMOTE






### Library Package

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn import datasets as dat
from sklearn import tree
from sklearn import metrics as met
from sklearn import model_selection as mod
from sklearn import linear_model as lin
from sklearn import preprocessing as pre
from sklearn import pipeline as pip # sklearn pipeline
from imblearn import pipeline as imbPip # imblearn pipeline
from imblearn import under_sampling as und
from imblearn import over_sampling as ove

### Algorithms

In [ ]:
rus = und.RandomUnderSampler(sampling_strategy=0.2, random_state=42)
X_res, y_res = rus.fit_resample(X, y)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
pd.Series(y_res).value_counts()

In [ ]:
pipe = imbPip.Pipeline([
      ("resample", ove.RandomOverSampler(sampling_strategy=0.3, random_state=42)),
      ("scaler", pre.StandardScaler()),
      ("sgd", lin.SGDClassifier(random_state=42))
])

mod.cross_val_score(pipe, X, y, cv=3, scoring="f1").mean()

In [ ]:
tl = und.TomekLinks(sampling_strategy="all")
X_res, y_res = tl.fit_resample(X, y)

pd.Series(y_res).value_counts()

In [ ]:
pipe = imbPip.Pipeline([
      ("resample", ove.SMOTE(sampling_strategy=0.3, k_neighbors=5, random_state=42)),
      ("scaler", pre.StandardScaler()),
      ("sgd", lin.SGDClassifier(random_state=42))
])

mod.cross_val_score(pipe, X, y, cv=3, scoring="f1").mean()

In [ ]:
# sampling techniques
for sampler in [und.RandomUnderSampler(random_state=42),
                ove.RandomOverSampler(random_state=42),
                ove.SMOTE(random_state=42)]:
  pipe_sampler_sgd = imbPip.Pipeline([
        ("resample", sampler),
        ("scaler", pre.StandardScaler()),
        ("sgd", lin.SGDClassifier(random_state=42))
  ])

  params = {
      "resample__sampling_strategy": [0.35, 0.4],
      "sgd__alpha": [0.001, 0.005],
      "sgd__max_iter": [500, 1000, 2000]
  }

  grid_sampler_sgd = mod.GridSearchCV(pipe_sampler_sgd, params, cv=3, scoring="roc_auc")
  grid_sampler_sgd.fit(X_train, y_train)

  print("GridSearchCV:", sampler.__class__.__name__, grid_sampler_sgd.best_score_)

  y_scores = grid_sampler_sgd.best_estimator_.decision_function(X_test)
  fpr, tpr, thresholds = met.roc_curve(y_test, y_scores)
  plt.plot(fpr, tpr, label=sampler.__class__.__name__)

# class weight tech
pipe_classw_sgd = pip.Pipeline([
      ("scaler", pre.StandardScaler()),
      ("sgd", lin.SGDClassifier(random_state=42))
])

params = {
    "sgd__alpha": [0.001, 0.005],
    "sgd__max_iter": [500, 1000, 2000],
    "sgd__class_weight": ["balanced", {0:1, 1:2}, {0:1, 1:3}]
}

grid_classw_sgd = mod.GridSearchCV(pipe_classw_sgd, params, cv=3, scoring="roc_auc")
grid_classw_sgd.fit(X_train, y_train)
print("GridSearch Class Weight:", grid_classw_sgd.best_score_)

y_scores = grid_classw_sgd.best_estimator_.decision_function(X_test)
fpr, tpr, thresholds = met.roc_curve(y_test, y_scores)
plt.plot(fpr, tpr, label="class_weight")

plt.plot([0, 1], [0, 1], "r--")
plt.legend()
plt.show()

## SVM Classification

*   Scale your data before using SVMs.

*   Nonlinear SVM Classification, some problems are not linearly separable so one approach might be adding Polynomial Features, and this operation can make the dataset linearly separable.
   - Another technique to tackle nonlinear problems is to add features computed using a similarity function that measures how much each instance resembles a particular landmark.






**Note :** Gaussian RBF kernel, If your model is overfitting, you should reduce gamma, and if it is underfitting, you should increase it (similar to the C hyperparameter).

### Library Package

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn import datasets as dat
from sklearn import tree
from sklearn import metrics as met
from sklearn import model_selection as mod
from sklearn import linear_model as lin
from sklearn import preprocessing as pre
from sklearn import pipeline as pip # sklearn pipeline
from sklearn import svm

### Algorithms

In [ ]:
#SVMs with Linear

pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("clf", svm.LinearSVC(loss="hinge", C=1))
])

mod.cross_val_score(pipe, X, y, cv=3).mean()

In [ ]:
# SVMs with Polynomial Features

pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("svm", svm.SVC(kernel="poly", degree=3, coef0=1, C=1, random_state=42, max_iter=5000))
])

mod.cross_val_score(pipe, X, y, cv=3, scoring="accuracy").mean()

In [ ]:
# SVMs with Gaussian RBF kernel

pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("svm", svm.SVC(kernel="rbf", gamma=1, C=1, random_state=42, max_iter=5000))
])

mod.cross_val_score(pipe, X, y, cv=3, scoring="accuracy").mean()

## Missing Value Imputation

### Library Package

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn import datasets as dat
from sklearn import tree
from sklearn import metrics as met
from sklearn import model_selection as mod
from sklearn import linear_model as lin
from sklearn import preprocessing as pre
from sklearn import pipeline as pip # sklearn pipeline
from sklearn import svm
from sklearn import impute as imp
from sklearn.experimental import enable_iterative_imputer
from sklearn import compose as com

### Algorithms

In [ ]:
pd.DataFrame(X).isnull().sum()

In [ ]:
x_values, y_values = [], []

In [ ]:
# Simple Imputer
for s in ["mean", "median"]:
  pipe = pip.Pipeline([
      ("scaler", pre.StandardScaler()),
      ("imp", imp.SimpleImputer(strategy=s)),
      ("sgd_reg", lin.SGDRegressor(random_state=42))
  ])

  score = mod.cross_val_score(pipe, X, y, scoring="neg_root_mean_squared_error").mean() * -1

  y_values.append(score)
  x_values.append("SimpleImp_" + s)

In [ ]:
# KNN Imputer

for w in ["uniform", "distance"]:
  for k in [5, 22]:
    pipe = pip.Pipeline([
        ("scaler", pre.StandardScaler()),
        ("imp", imp.KNNImputer(n_neighbors=k, weights=w)),
        ("sgd_reg", lin.SGDRegressor(random_state=42))
    ])

    score = mod.cross_val_score(pipe, X, y, scoring="neg_root_mean_squared_error").mean() * -1

    y_values.append(score)
    x_values.append("KNNImp_" + w + "_" + str(k))

In [ ]:
for est in [lin.SGDRegressor(random_state=42),
            lin.BayesianRidge(),
            tree.DecisionTreeRegressor(random_state=42)]:
  pipe = pip.Pipeline([
      ("scaler", pre.StandardScaler()),
      ("imp", imp.IterativeImputer(estimator=est, max_iter=5)),
      ("sgd_reg", lin.SGDRegressor(random_state=42))
  ])

  score = mod.cross_val_score(pipe, X, y, scoring="neg_root_mean_squared_error").mean() * -1

  y_values.append(score)
  x_values.append("IterativeImp_" + est.__class__.__name__)



plt.bar(x_values, y_values)
plt.ylim(0.7, 0.9)
plt.xticks(rotation=80)
plt.grid(True)
plt.show()

In [ ]:
pipe_num = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("imp", imp.SimpleImputer(strategy="mean"))
])

pipe_num2 = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("imp", imp.KNNImputer())
])

pipe_cat = pip.Pipeline([
    ("imp", imp.SimpleImputer(strategy="most_frequent")),
    ("one_hot", pre.OneHotEncoder(handle_unknown="ignore"))
])

#np.set_printoptions(suppress=True)
#pipe_num.fit_transform(data.select_dtypes(exclude="object"))

ct = com.ColumnTransformer([
    ("num", pipe_num, ["age"]),
    ("num2", pipe_num, ["salary"]),
    ("cat", pipe_cat, ["city"]),
])

ct.fit_transform(data)

In [ ]:
ct.transform(data_test)

In [ ]:
pipe_all = pip.Pipeline([
    ("ct", ct),
    ("sgd", lin.SGDClassifier())
])

pipe_all.fit(data, [0, 1, 1, 0])

In [ ]:
#second WAY 

pipe_cat = pip.Pipeline([
    ("encoding", pre.OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ("scaler", pre.StandardScaler())
])

ct = com.ColumnTransformer([
    ("cat", pipe_cat, ["Regional indicator"]), # 10
    ("num", pre.StandardScaler(), X_train.select_dtypes(exclude="object").columns) # 17
])

ct.fit_transform(X_train).shape

## Dimensionality Reduction

*   **Linear Projection**
    - Principal Component Analysis (PCA)
        * Feature scaling is required
        * PCA finds a low-dimensional representation of the data
        * It focuses the correlation among the features. 
        * Its combining the highly correlated features and represent this data with a smaller number of linearly uncorrelated features.
        * PCA finds the directions of maximum variance in the original high-dimensional data and projecting them onto a smaller dimensional space.

    - Singular Value Decomposition (SVD)

*   **Manifold Learning (Nonlinear Dimensionality Reduction**
    - T-distributed Stochastic Neighbor Embedding (t-SNE)
        * t-SNE is a nonlinear dimensionality reduction technique for visualizing high-dimensional data.
        * t-SNE minimizes the **Kullback–Leibler divergence** between the two probability distributions.



**NOTE :** Normal PCA, incremental PCA linearly project the original data onto a lower dimensional space, but there is also a nonlinear form of PCA known as kernel PCA, which runs a similarity function over pairs of original data points in order to perform nonlinear dimensionality reduction.

For the kernel PCA algorithm, we need to set:
*   the number of components
*   the type of kernel
*   the kernel coefficient, which is known as the gamma.

A practical tip - t-SNE:
*   First apply PCA to get around 50 features from N features (N >> 50)
*   Then, apply t-SNE to get 2-3 features from 50 features

### Library Package

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

from sklearn import datasets as dat
from sklearn import ensemble as ens
from sklearn import pipeline as pip
from sklearn import preprocessing as pre
from sklearn import decomposition as dec
from sklearn import manifold as man
from sklearn import model_selection as mod
from sklearn import linear_model as lin

### Algorithms

In [ ]:
data = dat.fetch_openml("mnist_784")

In [ ]:
X = data.data.values
y = data.target.values

In [ ]:
sns.scatterplot(x=X[:, 250], y=X[:, 366], hue=y)

In [ ]:
est = ens.RandomForestClassifier(n_estimators=10)
est.fit(X, y)

In [ ]:
np.set_printoptions(suppress=True)
est.feature_importances_.argsort()[::-1][:2]

In [ ]:
sns.scatterplot(x=X[:, 569], y=X[:, 350], hue=y)

In [ ]:
pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("pca", dec.PCA(n_components=2)) # unsupervised learning method
])

X_new = pipe.fit_transform(X) # X.shape -> (70000, 784) ==> X_new.shape -> (70000, 2)
X_new.shape

In [ ]:
pipe.named_steps["pca"].components_.shape

In [ ]:
pipe.named_steps["pca"].explained_variance_ratio_

In [ ]:
pipe.named_steps["pca"].explained_variance_ratio_.sum()

In [ ]:
sns.scatterplot(x=X_new[:, 0], y=X_new[:, 1], hue=y)

In [ ]:
pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("pca", dec.PCA(random_state=42))
])

pipe.fit(X)

In [ ]:
ratios = pipe.named_steps["pca"].explained_variance_ratio_

x_values, y_values = [], []

for i in range(1, 784):
  var = ratios[:i].sum()
  y_values.append(var)
  x_values.append(i)

plt.bar(x_values, y_values)
plt.show()

In [ ]:
ratios[:250].sum()

In [ ]:
pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("pca", dec.PCA(n_components=300, random_state=42)),
    #("sgd", lin.SGD...)
])

In [ ]:
pipe = pip.Pipeline([
    ("scaler", pre.StandardScaler()),
    ("pca", dec.PCA(n_components=100, random_state=42)),
    ("tsne", man.TSNE(n_components=2, random_state=42)) # unsupervised learning method
])

X_tsne = pipe.fit_transform(X[:5000])

In [ ]:
X_tsne.shape # 5000, 2

In [ ]:
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y[:5000])

In [ ]:
pipe.named_steps["pca"].components_.shape

In [ ]:
for i in range(10):
  sns.heatmap(pipe.named_steps["pca"].components_[i].reshape(28, 28))
  plt.show()

# Machine Learning - Unsupervised Learning By OV

Dimensionality reduction 
*   Matrix factorization: PCA, SVD
*   Manifold learning: MDS, t-SNE, UMAP



Clustering
* Hierarchical
* K-Means
* DBSCAN
* GMM

* Hierarchical clustering doesn’t require defining number of
clusters in advance, instead builds dendrogram
* It’s intuitive since we tend to group or split items based
on their similarities
* Time complexity is not ideal since it requires pairwise
operation O(n2)
* Linkage method has an influence on outcomes

Internal Measures: Choosing K by using the **average SILHOUETTE WIDTH**

Network properties
* Degree: # of connections
with other nodes
* Strength: sum of weights
attached to a node
* Density: fraction between #
of existing ties and

## Algorithms

### Library Package

In [ ]:
# Let's import some of the necessary libraries
import gzip
import json
import glob
import numpy as np
import networkx as nx

import matplotlib as mpl
from matplotlib import pyplot as plt

import pandas as pd


DATA_PATH = '/content/'
#DATA_PATH = '/content/'

# https://pypi.org/project/watermark/
!pip install watermark
!pip install packagename==0.0.1

import watermark
%load_ext watermark
%watermark -n -v -m -g -iv

### Algorithms

In [ ]:
ivaaData = {
    "Haci S.": [["Evrim K.", "Erdal K.", "Atif B."], "AkcanSA"],
    "Enes G.": [["Onur A.", "Cumali K.", "Nurullah B."], "EnerjiSA"],
    "Cemil C.": [["Ozge A.", "Volkan Z.", "Cumali K."], "EnerjiSA"],
    "Erdal K.": [["Ensari Y.", "Haci S.", "Atif B."], "TemSA"],
    "Tugce A.": [["Beste B.", "Erhan T.", "Selda O."], "AgeSA"],
    "Atif B.": [["Evrim K.", "Erdal K.", "Haci S."], "EnerjiSA-U"],
    "Beste B.": [["Tugce A.", "Nurullah B.", "Erhan T."], "BriSA"],
    "Asli C.": [["Irem E.", "Cumali K.", "Onur A."], "EnerjiSA"],
    "Volkan Z.": [["Cemil C.", "Ozge A.", "Cumali K."], "EnerjiSA"],
    "Irem E.": [["Asli C.", "Onur A.", "Cumali K."], "EnerjiSA"],
    "Ozge A.": [["Cemil C.", "Volkan Z.", "Enes G."], "EnerjiSA"],
    "Evrim K.": [["Haci S.", "Atif B.", "Ensari Y."], "AkcanSA"],
    "Cumali K.": [["Onur A.", "Irem E.", "Asli C."], "CimSA"],
    "Onur A.": [["Cumali K.", "Asli C.", "Irem E."], "EnerjiSA"],
    "Selda O.": [["Erhan T.", "Tugce A.", "Beste B."], "EnerjiSA"],
    "Ensari Y.": [[], "EnerjiSA"],
    "Nurullah B.": [[], "KordSA"],
    "Erhan T.": [[], "EnerjiSA"]
}

In [ ]:
print(len(ivaaData))
uniqueNames = set()
for k,v in ivaaData.items():
  uniqueNames.add(k)
  uniqueNames |= set(v[0])
print(len(uniqueNames))

In [ ]:
ivaaNet = nx.DiGraph()

for nodeA,neig in ivaaData.items():
  for nodeB in neig[0]:
    #print(nodeA, nodeB)
    ivaaNet.add_edge(nodeA, nodeB)

  ivaaNet.nodes[nodeA]['company'] = neig[1]

nx.info(ivaaNet)

In [ ]:
dict(ivaaNet.in_degree())

In [ ]:
betweennessC = nx.betweenness_centrality(ivaaNet)
for n in sorted(betweennessC, key=betweennessC.get, reverse=True):
    print(n, betweennessC[n])


In [ ]:
#pos = nx.spring_layout(ivaaNet)
#nx.draw_networkx(ivaaNet, pos=pos)

nx.write_gexf(ivaaNet, 'ivaaNet.gexf')

# Building Methods

In [ ]:
def reading_data(path, sep ):
  df_data = pd.read_csv(path,sep=sep)
  df_data.head()

In [ ]:
def creating_New_feature(feature_Data):
  new_fea = 